Creación de la base de datos de Personajes Mitológicos scrapeados de Wikipedia.

In [1]:
import numpy as np
import pandas as pd

import re
import unicodedata

import requests
from bs4 import BeautifulSoup

Función para obtener el objeto *BeautifulSoup* de la url.

In [2]:
def get_url(url):
    req = requests.get(url)
    if req.status_code == 200:
        document = BeautifulSoup(req.text, "html.parser")
    else:
        document = None
    return document

## <span style="color:green">Obtención URLs de categorías</span>

Primero se obtiene un dataset con todas las URLs de las categorías que scrapearemos más adelante.

### Obtención de mitologías

El conjunto de mitologías se obtiene de diferentes formas:

1. [Hijos (nodos inferiores) de la categoría "Mitología por cultura"](https://es.wikipedia.org/wiki/Especial:%C3%81rbolDeCategor%C3%ADas?target=Mitolog%C3%ADa_por_cultura&mode=all)
2. [Anexo de deidades](https://es.wikipedia.org/wiki/Anexo:Deidades)
3. Mitologías añadidas directamente a la lista

#### Mitologías 1

In [3]:
url = "https://es.wikipedia.org/wiki/Especial:ÁrbolDeCategorías?target=Mitología_por_cultura&mode=all"

In [4]:
pag = get_url(url)
nodos = pag.find("div", class_ = "CategoryTreeChildren").contents

In [5]:
mitologias1 = []
for nodo in nodos[3:]: # los primeros 3 elementos no contienen mitologias
    mitologias1.append(nodo.find("a").text)

In [6]:
mitologias1

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

#### Mitologías 2

In [7]:
url = "https://es.wikipedia.org/wiki/Anexo:Deidades"

In [8]:
pag = get_url(url)
items = [item.text for item in pag.find("div", id = "toc").find_all("span", class_ = "toctext")]

In [9]:
mitologias2 = []
for item in items:
    if item[:10] == "Mitología ":
        mitologias2.append(item)

In [10]:
mitologias2

['Mitología abenaki',
 'Mitología aborigen australiana',
 'Mitología judaica',
 'Mitología del cristianismo',
 'Mitología del catolicismo',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología árabe',
 'Mitología ashanti',
 'Mitología azteca',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología budista',
 'Mitología bushongo',
 'Mitología celta',
 'Mitología china',
 'Mitología chippewa',
 'Mitología coreana',
 'Mitología creek',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología dinka',
 'Mitología efik',
 'Mitología egipcia',
 'Mitología escandinava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología frigia',
 'Mitología griega',
 'Mitología guanche',
 'Mitología guaraní',
 'Mitología haida',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología huron',
 'Mitología igbo',
 'Mitología inca',
 'Mitología inuit',
 'Mitología iroquesa',
 'Mitología isoko',
 'Mitología japonesa',
 'Mitología khoikhoi',
 'Mitología kongo',
 'Mitología kurumba',
 'Mito

#### Mitologías 3

Se añaden 12 mitologías fuera de la lista.  
Además se añade la excepción de "Mitología del hinduismo".

In [11]:
# lista inicial
mitologias3 = ["Mitología ainu", "Mitología tlaxcalteca", "Mitología mixteca", "Mitología olmeca", "Mitología purépecha",
               "Mitología tolteca", "Mitología galesa", "Mitología rusa", "Mitología ucraniana", "Mitología siberiana",
               "Mitología persa", "Mitología huilliche", "Mitología acadia", "Mitología asiria", "Mitología babilónica",
               "Mitología vudú", "Mitología africana", "Mitología americana", "Mitología asiática", "Mitología europea",
               "Mitología oceánica", "Mitología fenicia", "Mitología semítica", "Mitología nubia", "Mitología del hinduismo",
               "Mitología judía"]
# lista final sabiendo las mitologias que tienen alguna categoria
mitologias3 = ["Mitología ucraniana", "Mitología siberiana", "Mitología persa", "Mitología vudú", "Mitología africana",
               "Mitología americana", "Mitología asiática", "Mitología europea", "Mitología oceánica", "Mitología fenicia",
               "Mitología semítica", "Mitología nubia", "Mitología del hinduismo"]

#### Agregación de las mitologías

In [12]:
mitologias = list(pd.unique(mitologias1 + mitologias2 + mitologias3))

In [13]:
mitologias

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

### Obtención de las URLs

Las URLs de las categorías están están formadas por (1) el *tipo de categoría* y (2) la *mitología*.

Ejemplo: `https://es.wikipedia.org/wiki/Categoría:Dioses_griegos` , donde `Dioses` es la categoría y `griegos` es la mitología.

1. Existen diferentes categorías que contienen personajes mitológicos (Deidades, Dioses, Gigantes, Demonios, Dragones ...).  
    Cada mitología tiene sus propias categorías, por lo que he elegido las categorías que he considerado compartidas entre mitologías. Estas categorías son: Personaje, Deidad, Dios, Diosa, Criatura, Dragón, Gigante, Demonio, Monstruo y Héroe.  
    Por otro lado, cada categoría puede tener diversos formatos según la mitología en particular. Por ejemplo, la categoría de "Criatura" puede estar formateada como: `Criaturas`, `Criaturas_mitológicas` y `Criaturas_de_la_mitología`.

In [14]:
# anteriormente se han probado más formatos
# algunos se añaden como excepción al tratarse de formatos únicos de cada mitologia
categs = {"Personajes_de_la_mitología" : "Personaje", "Deidades" : "Deidad", "Dioses_de_la_mitología" : "Dios",
          "Dioses" : "Dios", "Diosas_de_la_mitología" : "Diosa", "Diosas" : "Diosa", "Criaturas_de_la_mitología" : "Criatura",
          "Criaturas_mitológicas" : "Criatura", "Dragones" : "Dragón", "Gigantes_de_la_mitología" : "Gigante"}

2. El formato de la mitología depende tanto del tipo/formato de la categoría como de la propia mitología.  
    Por ejemplo, en la mitología egipcia podemos tener `Dioses_egipcios`, `Diosas_egipcias` y `Deidades_de_la_mitología_egipcia`. En cada caso, el formato de la mitología es diferente.  
    Además, cada mitología tiene su particular transformación cuando cambia de género y número. Los sustantivos del español suelen seguir unas reglas para esta transformación, aunque existen excepciones. Se aplican las 5 reglas más comunes a las mitologías para obtener sus términos clave, por lo que probablemente una de estas claves será correcta.  
    Algunas URLs tienen mayúscula la primera letra de la mitología, por lo que añado estas posibilidades a las claves.

Función para obtener los términos clave de una mitología.

In [15]:
def clave_mitologia(mit):
    lista = []
    lista.append("_".join(mit.split()[1:])) # elemento 0 en minuscula
    lista.append("_".join(mit.split()[1:-1] + [mit.split()[-1].capitalize()])) # elemento 1 en mayuscula
    for i in range(2):
        lista.append(lista[i] + "s")
        lista.append(lista[i] + "es")
        lista.append(lista[i][0:-1] + "os")
        lista.append(lista[i][0:-1] + "es")
    return(lista)

Función para comprobar si la URL de la categoría contiene personajes.

In [16]:
def url_buena(url):
    doc = get_url(url)
    if doc != None:
        paginas = doc.find("div", id = "mw-pages")
        if paginas != None:
            return(True)
    return(False)

Función para obtener las URLs de una mitología.

In [17]:
def obtener_urls(mit):
    url_base = "https://es.wikipedia.org/wiki/Categoría:{}_"
    urls = pd.DataFrame()
    keys = clave_mitologia(mit)
    formatos = list(categs.keys())
    for formato in formatos:
        for key in keys:
            url = url_base.format(formato) + key
            if url_buena(url):
                fila = pd.DataFrame([[url,
                                     mit,
                                     categs[formato]]])
                urls = urls.append(fila)
    return(urls.drop_duplicates(ignore_index = True))

Se combinan los tipos de categoría y las mitologías y se obtiene un dataset con las categorías existentes en Wikipedia.

**Tarda mucho en ejecutarse**

In [18]:
urls_categorias = pd.DataFrame()

for mitologia in mitologias:
    urls_categorias = urls_categorias.append(obtener_urls(mitologia))

urls_categorias.columns = ["URL", "Mitología", "Clase"]

Se añaden directamente las siguientes categorías con formato único:
* [Dioses primordiales de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega)
* [Titanes (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Titanes)
* [Deidades de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega)
* [Dioses olímpicos menores de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores)
* [Héroes de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega)
* [Monstruos de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega)
* [Musas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Musas)
* [Oceánidas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánidas)
* [Oceánides (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánides)
* [Descendientes de Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus)
* [Descendientes de Poseidón (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón)
* [Descendientes de Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo)
* [Descendientes de Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita)
* [Diosas tomadas por Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus)
* [Diosas amadas por Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo)
* [Dioses que tomaron a Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita)
* [Jotuns (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Jotuns)
* [Vanir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Vanir)
* [Æsir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Æsir)
* [Ásynjur (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Ásynjur)
* [Nornas (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Nornas)
* [Lobos de la mitología nórdica](https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica)
* [Hijos de Odín (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín)
* [Demonios de la mitología hinduista](https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo)
* [Divinidades sintoistas (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas)
* [Yōkai (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Yōkai)
* [Tonalteteuctin (mitología mexica)](https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin)
* [Dioses del viento de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica)
* [Criaturas de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas)
* [Demonios de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica)
* [Dharmapalas (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dharmapalas)
* [Dakinis (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dakinis)
* [Deidades del budismo tibetano](https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano)

In [20]:
addCategs = [["https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Titanes","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores","Mitología griega", "Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega","Mitología griega","Héroe"],
             ["https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega","Mitología griega","Monstruo"],
             ["https://es.wikipedia.org/wiki/Categoría:Musas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánidas","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánides","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Jotuns","Mitología nórdica","Gigante"],
             ["https://es.wikipedia.org/wiki/Categoría:Vanir","Mitología nórdica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Æsir","Mitología nórdica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Ásynjur","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nornas","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín","Mitología nórdica","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo","Mitología del hinduismo","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas","Mitología japonesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Yōkai","Mitología japonesa","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas","Mitología mexica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica","Mitología mexica","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Dharmapalas","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dakinis","Mitología budista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano","Mitología budista","Deidad"]]

urls_categorias = urls_categorias.append(pd.DataFrame(addCategs, columns = ["URL", "Mitología", "Clase"]))

Se añaden las URLs de las categorías con más de una página de personajes.

Función para obtener la URL de la siguiente página de personajes en el caso de que exista.

In [21]:
def obtener_siguiente(pagina):
    first_link = pagina.find("div", id = "mw-pages").find("a")
    if first_link.text == "página siguiente":
        return "https://es.wikipedia.org" + first_link.get("href")
    elif first_link.text == "página anterior":
        second_link = pagina.find("div", id = "mw-pages").find_all("a")[1]
        if second_link.text == "página siguiente":
            return "https://es.wikipedia.org" + second_link.get("href")

In [22]:
urls_categorias = urls_categorias.reset_index(drop = True)

for i in range(len(urls_categorias)):
    next_url = obtener_siguiente(get_url(urls_categorias["URL"][i]))
    while next_url is not None:
        new_categoria = pd.DataFrame([[next_url, urls_categorias["Mitología"][i], urls_categorias["Clase"][i]]])
        new_categoria.columns = ["URL", "Mitología", "Clase"]
        urls_categorias = urls_categorias.append(new_categoria)
        next_url = obtener_siguiente(get_url(next_url))

In [23]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología árabe,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología árabe,Diosa
2,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología cananea,Dios
3,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología celta,Deidad
4,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología celta,Dios
...,...,...,...
106,https://es.wikipedia.org/wiki/Categoría:Dharma...,Mitología budista,Deidad
107,https://es.wikipedia.org/wiki/Categoría:Dakinis,Mitología budista,Diosa
108,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología budista,Deidad
0,https://es.wikipedia.org/w/index.php?title=Cat...,Mitología griega,Personaje


In [24]:
urls_categorias.to_csv("urls_categorias.csv", header = False, index = False)

## Obtención URLs de personajes

Ahora se obtiene un dataset con las URLs de los personajes pertenecientes a las categorías extraídas en el apartado anterior. También se añaden algunos personajes importantes manualmente.

In [3]:
urls_categorias = pd.read_csv("urls_categorias.csv", names = ["URL", "Mitología", "Clase"])
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología árabe,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología árabe,Diosa
2,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología cananea,Dios
3,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología celta,Deidad
4,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología celta,Dios
...,...,...,...
106,https://es.wikipedia.org/wiki/Categoría:Dharma...,Mitología budista,Deidad
107,https://es.wikipedia.org/wiki/Categoría:Dakinis,Mitología budista,Diosa
108,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología budista,Deidad
109,https://es.wikipedia.org/w/index.php?title=Cat...,Mitología griega,Personaje


Función para limpiar el nombre del personaje.

In [4]:
def clean_name(nombre):
    if nombre.find("mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("Mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("(dios") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("deidad") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("personaje") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    else:
        return(nombre)

Función para obtener los nombres y las URLs de los personajes de la categoría.

La división alfabética de los personajes puede estar en dos formatos diferentes:  

* Todas las categorias en el mismo nodo
* Cada categoria tiene su nodo

In [5]:
def scrap_categoria(url):
    pagina = get_url(url)
    lista_pers = []
    lista_urls = []
    div_alfabetica = pagina.find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr")
    if div_alfabetica.find("div", class_ = "mw-category") is None: # caso 1
        categorias = div_alfabetica.find_all("h3")
        check_letras = [categoria.text.isalpha() for categoria in categorias]
        personajes = div_alfabetica.find_all("ul")
        for i in range(len(categorias)):
            if check_letras[i]:
                pers = personajes[i]
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})
    else: # caso 2
        categorias = div_alfabetica.find_all("div", class_ = "mw-category-group")
        for categoria in categorias:
            if categoria.find("h3").text.isalpha():
                pers = categoria.find_all("li")
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})

Se obtienen los personajes de las categorías con sus URLs.

In [6]:
urls_personajes = pd.DataFrame()

for url in urls_categorias["URL"]:
    urls_personajes = urls_personajes.append(scrap_categoria(url))

Algunos personajes mitológicos importantes no pertenecen a ninguna categoría, por lo que se añaden directamente:  

*Mitología griega*
* [Tánatos](https://es.wikipedia.org/wiki/Tánatos)
* [Zeus](https://es.wikipedia.org/wiki/Zeus)
* [Poseidón](https://es.wikipedia.org/wiki/Poseidón)
* [Apolo](https://es.wikipedia.org/wiki/Apolo)
* [Atenea](https://es.wikipedia.org/wiki/Atenea)
* [Artemisa](https://es.wikipedia.org/wiki/Artemisa)
* [Deméter](https://es.wikipedia.org/wiki/Deméter)
* [Heracles](https://es.wikipedia.org/wiki/Heracles)
* [Teseo](https://es.wikipedia.org/wiki/Teseo)
* [Jasón](https://es.wikipedia.org/wiki/Jasón)

*Mitología romana*
* [Mitra](https://es.wikipedia.org/wiki/Mitra_(dios_romano))
* [Magna Mater](https://es.wikipedia.org/wiki/Magna_Mater)

*Mitología nórdica*
* [Hela](https://es.wikipedia.org/wiki/Hela)

*Mitología mesopotámica*
* [Gilgamesh](https://es.wikipedia.org/wiki/Gilgamesh)
* [Tiamat](https://es.wikipedia.org/wiki/Tiamat)
* [Marduk](https://es.wikipedia.org/wiki/Marduk)
* [Sin](https://es.wikipedia.org/wiki/Sin)
* [Shamash](https://es.wikipedia.org/wiki/Shamash)
* [Kur](https://es.wikipedia.org/wiki/Kur)

*Mitología china*
* [Fuxi](https://es.wikipedia.org/wiki/Fuxi)

*Mitología incaica*
* [Inti](https://es.wikipedia.org/wiki/Inti)
* [Illapa](https://es.wikipedia.org/wiki/Illapa)
* [Cuchavira](https://es.wikipedia.org/wiki/Cuchavira)

*Mitología pueblo*
* [Kokopelli](https://es.wikipedia.org/wiki/Kokopelli)
* [Awelo](https://es.wikipedia.org/wiki/Awelo)

In [7]:
addPers = [["https://es.wikipedia.org/wiki/Tánatos","Tánatos","Mitología griega","Deidad"],
           ["https://es.wikipedia.org/wiki/Zeus","Zeus","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Poseidón","Poseidón","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Apolo","Apolo","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Atenea","Atenea","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Artemisa","Artemisa","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Deméter","Deméter","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Heracles","Heracles","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Teseo","Teseo","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Jasón","Jasón","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Mitra_(dios_romano)","Mitra","Mitología romana","Dios"],
           ["https://es.wikipedia.org/wiki/Magna_Mater","Magna Mater","Mitología romana","Diosa"],
           ["https://es.wikipedia.org/wiki/Hela","Hela","Mitología nórdica","Deidad"],
           ["https://es.wikipedia.org/wiki/Gilgamesh","Gilgamesh","Mitología mesopotámica","Héroe"],
           ["https://es.wikipedia.org/wiki/Tiamat","Tiamat","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Marduk","Marduk","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Sin","Sin","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Shamash","Shamash","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Kur","Kur","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Fuxi","Fuxi","Mitología china","Dios"],
           ["https://es.wikipedia.org/wiki/Inti","Inti","Mitología incaica","Dios"],
           ["https://es.wikipedia.org/wiki/Illapa","Illapa","Mitología incaica","Dios"],
           ["https://es.wikipedia.org/wiki/Cuchavira","Cuchavira","Mitología incaica","Dios"],
           ["https://es.wikipedia.org/wiki/Kokopelli","Kokopelli","Mitología pueblo","Dios"],
           ["https://es.wikipedia.org/wiki/Awelo","Awelo","Mitología pueblo","Dios"]]

urls_personajes = urls_personajes.append(pd.DataFrame(addPers, columns = ["URL", "Nombre", "Mitología", "Clase"]))
urls_personajes = urls_personajes.reset_index(drop = True)

Hay personajes repetidos, es decir, personajes que pertenecen a distintas Mitologías o a distintas Clases. Por lo tanto, cambiamos el formato de ambas variables ("Mitología" y "Clase") en One-Hot Encoding.

In [8]:
urls_personajes.groupby(by = ["URL", "Nombre"]).size().sort_values(ascending = False)

URL                                                            Nombre                  
https://es.wikipedia.org/wiki/El_(dios_sem%C3%ADtico)          El                          5
https://es.wikipedia.org/wiki/Elohim                           Elohim                      5
https://es.wikipedia.org/wiki/Baal                             Baal                        5
https://es.wikipedia.org/wiki/Yam_(dios)                       Yam                         5
https://es.wikipedia.org/wiki/Qilin                            Qilin                       4
                                                                                          ..
https://es.wikipedia.org/wiki/Menecio_(hijo_de_J%C3%A1peto)    Menecio (hijo de Jápeto)    1
https://es.wikipedia.org/wiki/Meneceo                          Meneceo                     1
https://es.wikipedia.org/wiki/Menae                            Menae                       1
https://es.wikipedia.org/wiki/Men_(dios)                       Men         

In [9]:
ohe = (pd
       .get_dummies(urls_personajes[["Mitología", "Clase"]],
                    prefix = {"Mitología" : "", "Clase" : "Clase"},
                    prefix_sep = {"Mitología" : "", "Clase" : "_"}))
urls_personajes = (pd.
        concat([urls_personajes, ohe], axis = 1).
        drop(columns = ["Mitología", "Clase", "Clase_Personaje"]).
        groupby(by = ["URL", "Nombre"]).
        agg(max).
        reset_index())
urls_personajes

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Mitología árabe,Clase_Criatura,Clase_Deidad,Clase_Demonio,Clase_Dios,Clase_Diosa,Clase_Dragón,Clase_Gigante,Clase_Héroe,Clase_Monstruo
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,https://es.wikipedia.org/wiki/%C3%81gave_(mito...,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2611,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2612,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2613,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2614,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


#### Limpieza

* quitar los que contengan "(desambiguación)"
* Cloris es "desambiguación"
* Ágave (mitología) es "desambiguación"
* Aletes (deidad) es un personaje historico, por lo que se quita

In [10]:
urls_personajes.groupby("Nombre").size().sort_values(ascending = False)

Nombre
Ágave                             2
Cloris                            2
Aletes                            2
Mitra                             2
Anu                               2
                                 ..
Nesoi                             1
Neso                              1
Nerthus                           1
Nerites                           1
Abante (compañero de Diomedes)    1
Length: 2611, dtype: int64

In [11]:
urls_personajes = urls_personajes[[not("desambiguación" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("mitología" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("dios" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("Guerra" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("Anexo" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/Cloris"]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/%C3%81gave_(mitolog%C3%ADa)"]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/Aletes_(deidad)"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses Olímpicos (DC Comics)"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses domésticos"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses egipcios"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Diosas Tenma"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Los Dioses Gemelos"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Siete Dioses de la Fortuna"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Deidades feroces"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Estatua de Sejmet"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Divinidades hinduistas"]

Anu es el nombre de un dios en la mitología mesopotámica y de otro dios en la mitologia celta. Pasa lo mismo con Mitra.

In [12]:
urls_personajes.loc[urls_personajes.URL == "https://es.wikipedia.org/wiki/Anu", "Nombre"] = "Anu (mitología mesopotámica)"
urls_personajes.loc[urls_personajes.URL == "https://es.wikipedia.org/wiki/Anu_(diosa_celta)", "Nombre"] = "Anu (mitología celta)"
urls_personajes.loc[urls_personajes.URL == "https://es.wikipedia.org/wiki/Mitra_(dios_indio)", "Nombre"] = "Mitra (mitología del hinduismo)"
urls_personajes.loc[urls_personajes.URL == "https://es.wikipedia.org/wiki/Mitra_(dios_romano)", "Nombre"] = "Mitra (mitología romana)"

In [13]:
urls_personajes.groupby("Nombre").size().sort_values(ascending = False)

Nombre
Šakkan                            1
Eurídice de Tebas                 1
Etra                              1
Etolo (hijo de Óxilo)             1
Etiäinen                          1
                                 ..
Nete                              1
Nesoi                             1
Neso                              1
Nerthus                           1
Abante (compañero de Diomedes)    1
Length: 2589, dtype: int64

In [14]:
urls_personajes

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Mitología árabe,Clase_Criatura,Clase_Deidad,Clase_Demonio,Clase_Dios,Clase_Diosa,Clase_Dragón,Clase_Gigante,Clase_Héroe,Clase_Monstruo
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2611,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2612,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2613,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2614,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
urls_personajes.to_csv("urls_personajes.csv", header = True, index = False)

## Obtención texto de personajes

In [20]:
data = pd.read_csv("urls_personajes.csv")
data

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Mitología árabe,Clase_Criatura,Clase_Deidad,Clase_Demonio,Clase_Dios,Clase_Diosa,Clase_Dragón,Clase_Gigante,Clase_Héroe,Clase_Monstruo
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2585,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2586,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2587,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


Obtenemos el texto de cada artículo.

Función para limpiar el texto.

In [17]:
def clean_text(text):
    text = re.sub("[^A-Za-zÀ-ÿ\u00f1\u00d1]", " ", text)
    text = text.lower()
    out = ""
    for word in text.split():
        out += " " + word
    return(out.strip())

Función para obtener todo el texto útil del artículo de wikipedia. Se omiten los apartados de "Enlaces externos", "Referencias", "Fuentes", "Notas", "Véase también" y "Bibliografía".

In [18]:
omit = ["enlaces externos", "referencias", "fuentes", "notas", "véase también", "bibliografía"]

def obt_texto(url):
    pag = get_url(url)
    nodos = pag.find("div", class_ = "mw-parser-output").contents
    texto = ""
    h2 = True
    h3 = True
    for i in range(len(nodos)):
        if isinstance(nodos[i], str):
            continue
        if nodos[i].get("class") is not None:
            if ["noprint", "VT rellink", "ext-quick-survey-panel"] in nodos[i].get("class") or "VT ":
                continue
        tipo = nodos[i].name
        if tipo == "h2":
            if len(set([nodos[i].text.lower().find(o) for o in omit])) == 1: # si el string no contiene ninguna de las palabras clave...
                h2 = True
                h3 = True
            else: # si el string contiene alguna de las palabras que se quieren omitir...
                h2 = False
        if tipo == "h3":
            if len(set([nodos[i].text.find(o) for o in omit])) == 1:
                h3 = True
            else:
                h3 = False
        if h2 and h3:
            texto += nodos[i].text
    return(clean_text(texto))

In [21]:
obt_texto(data["URL"][45])

'en la mitología griega aérope en griego antiguo cara brumosa fue una princesa cretense hija de catreo y nieta de minos su padre había recibido un oráculo acerca de que moriría a manos de uno de sus hijos pero lo mantuvo en secreto sin embargo su hijo altémenes lo descubrió y huyó a rodas con su hermana apemósine catreo entregó a aérope y a su hermana clímene al viajero nauplio para que las vendiera como esclavas en un país extranjero este las llevó a argos donde la primera fue comprada y desposada por plístenes el rey del país con quien fue madre de agamenón menelao y anaxibia otra tradición cuenta que catreo entregó a aérope a nauplio no solo por temor a morir sino porque ésta había tenido relaciones con un esclavo encargándole que la arrojase al mar según esta misma versión aérope casó con atreo quien habría sido el padre de agamenón menelao y anaxibia para conciliar ambas tradiciones algunas fuentes señalan que plístenes era el hijo o el padre de atreo y que a su muerte aérope casó

In [22]:
textos = []
for url in data["URL"]:
    textos.append(obt_texto(url))

data["Texto"] = textos

data

In [24]:
data

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Clase_Criatura,Clase_Deidad,Clase_Demonio,Clase_Dios,Clase_Diosa,Clase_Dragón,Clase_Gigante,Clase_Héroe,Clase_Monstruo,Texto
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en la mitología griega áctor era padre de mene...
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en la mitología griega áctor en griego antiguo...
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,en el marco del vedismo áditi es una diosa mad...
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en la mitología griega ágave en griego antiguo...
4,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,álfröðull alfrodhul del nórdico antiguo gloria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en la mitología griega zeuxipo griego antiguo ...
2585,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,zipacná era en la mitología maya hijo de vucub...
2586,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,zonget es la diosa de la naturaleza en la mito...
2587,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,en la mitología griega la zorra teumesia en gr...


In [25]:
data.to_csv("data.csv", header = True, index = False)

A la hora de obtener el texto de los artículos, sacar la matriz para crear el grafo.